## Introduction

In this notebook, we will demonstrate how to use the Solcast SDK to retrieve and visualize solar radiation data. We will start by loading the necessary libraries, then proceed to fetch historical data for a specific location, and finally plot the results using XPlot.Plotly.


In [1]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: System.Net.Http"
#r "nuget: XPlot.Plotly, 4.1.0"
#r "nuget:xplot.plotly.interactive"
#r "../../src/Solcast/bin/Debug/net8.0/Solcast.dll"

Installed Packages Newtonsoft.Json, 13.0.1 System.Net.Http, 4.3.4 XPlot.Plotly, 4.1.0 xplot.plotly.interactive, 4.1.0

Loading extensions from `/root/.nuget/packages/xplot.plotly.interactive/4.1.0/lib/net7.0/XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

To begin, we load the necessary libraries for using the Solcast SDK:

In [2]:
using System;
using System.Threading.Tasks;
using Solcast;
using Solcast.Clients;

To test the API you can use [unmetered locations](https://docs.solcast.com.au/#unmetered-locations), in the sdk:

In [3]:
var sydney = UnmeteredLocations.Locations["Sydney Opera House"];

Retrieve the historical data and plot the results.

In [5]:
using Newtonsoft.Json;
using System.Linq;
using XPlot.Plotly;


var historicClient = new HistoricClient();

double latitude = sydney.Latitude;
double longitude = sydney.Longitude;
string start = "2022-06-01T14:45:00.000Z";
string duration = "P5D";
string period = "PT5M";

var response = await historicClient.GetRadiationAndWeather(latitude, longitude, start, duration: duration, period: period, format: "json");

// Deserialize JSON into a dynamic object
dynamic data = JsonConvert.DeserializeObject(response);

// Extract data from dynamic object
var periodEnd = ((IEnumerable<dynamic>)data.estimated_actuals)
    .Select(d => (DateTime)d.period_end)
    .ToArray();

var ghi = ((IEnumerable<dynamic>)data.estimated_actuals)
    .Select(d => (double)d.ghi)
    .ToArray();

var clearskyGHI = ((IEnumerable<dynamic>)data.estimated_actuals)
    .Select(d => (double)d.dni)  // Using dni as clearsky_ghi
    .ToArray();

// Create traces for GHI and Clearsky GHI
var ghiTrace = new Scatter
{
    x = periodEnd,
    y = ghi,
    name = "GHI",
    mode = "lines",
//    line = new Line { Color = "orange" }
};

var clearskyGhiTrace = new Scatter
{
    x = periodEnd,
    y = clearskyGHI,
    name = "Clearsky GHI",
    mode = "lines",
//    line = new Line { Color = "blue" }
};

// Plot the chart
var chart = Chart.Plot(new[] { ghiTrace, clearskyGhiTrace });
chart.WithTitle("GHI vs Clearsky GHI");
chart.WithXTitle("Period End");
chart.WithYTitle("Radiation (W/m²)");

display(chart);


<!DOCTYPE html>